Tester et entraîner un modèle de reconnaissance d'écriture
===
<br>

Les dossiers qui constituent l'archive de la correspondance de Constance de Salm réunissent des documents écrits par plusieurs scribes. La différence des écritures représente une difficulté certaine pour la reconnaissance automatique.

Ce *notebook* explique comment procéder à la reconnaissance automatique de l'écriture dans ce contexte particulier impliquant de multiples écritures.

# Classer les images par mains

Inventorier toutes les mains attestées dans une source n'est pas toujours aisé.

L'objectif reste avant tout de repérer les mains principales : celles attestées sur le plus grand nombre de pages. Une main qui ne serait attestée que sur une dizaine de pages ne mériterait pas d'être classée, car entraîner un modèle de reconnaissance d'écriture pour un petit nombre de pages est une perte de temps.

Une fois les mains principales repérées, on renseigne le fichier [mains.csv](./sources/mains.csv) qui jouera un rôle  dans l'évaluation des modèles et leur entraînement.

# Créer un échantillon-test de chaque écriture

Afin de pouvoir tester un ou plusieurs modèles, il est nécessaire de constituer une vérité de terrain de 2-3 doubles pages (selon la densité d'écriture qu'elles contiennent).

## Critères de sélection

On crée dans le dossier de chaque main un dossier **test** contenant des spécimens d'écriture selon les critères suivants :
- Reproductions de bonne qualité (sans problème de transparence)
- Pages choisies de manière discontinue (l'écriture d'une même main peut en effet varier et il est utile de prendre en compte cette variété pour le test)

Si certaines mains ne sont attestées qu'en compagnie d'autres écritures, on veillera à limiter le test de reconnaissance d'écriture aux seuls lignes de la main à tester (en supprimant après segmentation les lignes non pertinentes afin que la reconnaissance de l'écriture ne les traite pas).

## Organisation des fichiers

Voici un modèle d'arborescence pour le rangement et le nommage des fichiers et dossiers :
```txt
sources/
├── mainCdS02_Konv002_01/
│   ├── test/
│   │   ├── CdS02_Konv002-02_0065.jpg
│   │   ├── CdS02_Konv002-02_0073.jpg
│   ├── train/
├── mainCdS02_Konv019_01/
│   ├── test/
│   │   ├── CdS02_Konv019_0002.jpg
│   │   ├── CdS02_Konv019_0003.jpg
│   └── train/
├── mains.csv
└── modeles/
```

## Critères de transcription

On suit de manière rigoureuse les préconisations pour la transcription énoncées dans la [documentation analytique](https://github.com/sbiay/CdS-edition/blob/main/documentation/documentation.pdf) (cf. annexe Normes de transcription).


# Tester des modèles HTR

## Installer l'application Kraken

On recommande pour tester un modèle d'utiliser l'application Kraken en ligne de commande, disponible pour Linux et Mac OSX (non pour Windows). Les instructions sont consultables [ici](https://github.com/mittagessen/kraken#installation).

## Importer un modèle

Les modèles extérieurs au projet que l'on a utilisés sont téléchargeables sur le [Gitlab du laboratoire Inria](https://gitlab.inria.fr/dh-projects/kraken-models/-/tree/master/transcription%20models) :
- generic_lectaurep_26.mlmodel
- cm_ft_mrs15_11.mlmodel

## Initier un journal de tests

Le script [journalReconn.py](./py/journalReconn.py) permet de pré-remplir un journal pour l'enregistrement des résultats des tests effectués sur les modèles. 

On lui donne comme argument un nom de modèle et il écrit dans le fichier Json [journal-rec.json](./sources/journal-rec.json), à la date et à l'heure courante et pour chaque main listée dans le fichier [mains.csv](./sources/mains.csv), les noms de ces mains et prépare le renseignement des valeurs de test.

Conseils d'utilisation :
- Paramètre MODELE : obligatoire ; reseigner un **nom de modèle** plutôt que son chemin relatif ou absolu
- Option -v, --veriteterrain : le test initial se fait naturellement sans avoir entraîné le modèle sur les vérités de terrain
- Option -i, --ignore : pour ignorer une main listée dans le fichier [mains.csv](./sources/mains.csv)

In [1]:
!python3 py/journalReconn.py MODELE

Le contenu écrit dans le fichier journal donne comme **accuracy** pour chaque main une valeur de 0. Cette valeur doit être saisie manuellement dans le fichier une fois effectué le test comme suit.


## Effectuer un test

Avec la commande suivante on effectue un test d'acuité pour un modèle sur une main particulière (on doit relever l'*Average accuracy*) :

In [9]:
# ketos test -m ../sources/modeles-rec/NOM-MODEL.mlmodel sources/NOM-MAIN/test/*xml -f alto
!ketos test -m ./sources/modeles-rec/cm_ft_mrs15_11.mlmodel sources/mainCdS02_Konv019_02/test/*xml -f alto

Loading model ./sources/modeles-rec/cm_ft_mrs15_11.mlmodel	✓
Evaluating ./sources/modeles-rec/cm_ft_mrs15_11.mlmodel
Evaluating ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 88/88 0:00:00 0:00:22 0:00:01 0:00:22
=== report  ===

4981	Characters
186	Errors
96.27%	Accuracy

38	Insertions
25	Deletions
123	Substitutions

Count	Missed	%Right
3946	127	96.78%	Latin
1035	34	96.71%	Common

Errors	Correct-Generated
15	{ é } - { e }
9	{ . } - {  }
9	{ a } - { u }
7	{ SPACE } - {  }
6	{  } - { e }
5	{ o } - { a }
4	{ n } - { u }
4	{ u } - { n }
4	{  } - { SPACE }
3	{ , } - {  }
3	{  } - { . }
3	{ r } - {  }
3	{ a } - { à }
3	{  } - { - }
3	{ SPACE } - { . }
3	{ a } - { o }
3	{ l } - { t }
2	{ è } - { i }
2	{ e } - { i }
2	{ x } - { 2 }
2	{ i } - { e }
2	{ l } - {  }
2	{ t } - { s }
2	{ S } - { s }
2	{ ç } - { c }
2	{ : } - {  }
2	{ V } - { v }
2	{ d } - {  }
2	{ r } - { t }
2	{ c } - { e }
2	{ t } - {  }
2	{ n } - { m }
2	{ q } - { g }
2	{ e } - { é }
1	{ , } - { SPACE }
1	{  } - { ; }
1	{ : } - {

Il convient de choisir le modèle présentant **les meilleurs résultats pour l'ensemble des écritures testées**. Si le meilleur modèle n'atteint pas 90% pour tout ou partie des mains, il convient de l'entraîner par l'apport de vérités de terrain pour chaque main n'atteignant pas ce score.

# Entraîner un modèle

On procède à la constitution d'une vérité de terrain pour chaque spécimen d'écriture. Il s'agit de transcrire l'**équivalent d'une dizaine de pages simples** : si les pages présentent des blancs importants ou d'autres écritures étrangères à l'entraînement (elles ne doivent pas être transcrites) on augmentera le nombre de pages pour parvenir grosso modo à ce volume de 10 pages simples.

La transcription de ces pages se fait à la main, sans l'aide d'une première reconnaissance de l'écriture, car il est plus compliqué de corriger une mauvaise prédiction que de transcrire manuellement.

On procède à la segmentation de la page (comme expliqué dans le notebook [suivant](./2_Realiser_une_prediction_HTR.ipynb)). On se contente d'une segmentation automatique, puis on transcrit les lignes manuellement.